IAPMEI

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

pd.set_option('display.float_format', lambda x: '%0.2f' % x)

Registos encerrados/anulados pós-contrato

In [ ]:
closed = pd.read_csv("APPIeFACIE-Resposta.csv", header = 0, encoding='latin-1')
nulled = pd.read_csv("Anulações-Resposta.csv", header = 0, encoding='latin-1')

In [ ]:
closed = closed[closed["Tipo"] != "PPI"]

In [ ]:
closed["Nproj_anon"].nunique()

In [ ]:
nulled["Nproj_anon"].nunique()

# General statistics of the finished/annulled projects

In [ ]:
closed["Tipo"].value_counts()

In [ ]:
closed.columns

In [ ]:
nulled["Tipo"].value_counts()

In [ ]:
nulled.groupby("Tipo")["Nproj_anon"].count().to_frame()

In [ ]:
nulled["Motivo"].value_counts().sum()

In [ ]:
nulled.columns

In [ ]:
nulled["Nproj_anon"].isna().sum()

In [ ]:
nulled.groupby(["Tipo", "Motivo"])["Nproj_anon"].count().to_frame()

In [ ]:
nulled[(nulled["Tipo"] == "Anulação pós-contrato") & (nulled["Motivo"].isna())]["Nproj_anon"].nunique()

### Filter data based on type of closure and annulment

All but "desistência do promotor" should be chosen

In [ ]:
nulled["Motivo"].value_counts()

In [ ]:
closed = closed[(closed["Tipo"] == "ENCINV") |
                (closed["Tipo"] == "ENCPRJ")]
nulled = nulled[(nulled["Tipo"] == "Anulação pós-contrato") &
               (nulled["Motivo"] != "Desistência do promotor")]

In [ ]:
closed["Nproj_anon"].nunique()

In [ ]:
nulled[(nulled["Tipo"] == "Anulação pós-contrato") &
               (nulled["Motivo"] != "Desistência do promotor")]

### Number of projects for each class of target feature

1. Closed
1. Cancelled

In [ ]:
print(closed["Nproj_anon"].nunique())
print(nulled["Nproj_anon"].nunique())

In [ ]:
projects = closed["Nproj_anon"].unique()
projects = np.append(projects, nulled["Nproj_anon"].unique())

In [ ]:
len(projects)

In [ ]:
len(np.unique(projects))

# Treating the several files to then merge in a final dataset

In [ ]:
folder = "dados/"

In [ ]:
application = pd.read_csv(folder+"candidaturas/Geral_candidaturas.csv", encoding='latin-1')
invoice = pd.read_csv(folder+"facie/Facturas_facie.csv", encoding='latin-1')
worker = pd.read_csv(folder+"ptcand/PTrabalho_candidaturas.csv", encoding='latin-1')
expenditure = pd.read_csv(folder+"appi/Inv_appi.csv", encoding='utf-8')
financing = pd.read_csv(folder+"candidaturas/Financ_candidaturas.csv", encoding='utf-8')
esf = pd.read_csv(folder+"candidaturas/FSE_candidaturas.csv", encoding='utf-8')

ies_1 = pd.read_csv(folder+"ies/ies_prom_coletivos_selected_1.csv", encoding='utf-8')
ies_2 = pd.read_csv(folder+"ies/ies_prom_coletivos_selected_2.csv", encoding='utf-8')
ies_3 = pd.read_csv(folder+"ies/ies_prom_singulares.csv", encoding='utf-8')

In [ ]:
application["N_Proj_anon"].nunique()

In [ ]:
application[application["N_Proj_anon"].isin(projects)]["N_Proj_anon"].nunique()


In [ ]:
ies_3[ies_3["N_Proj_anon"].isin(projects)]

In [ ]:
# add the features related to INE
ginind = pd.read_excel("INE/ginind.xlsx")
estemp = pd.read_excel("INE/estemp.xlsx")
denpop = pd.read_excel("INE/denpop.xlsx")
uninut = pd.read_excel("INE/uninut.xlsx")
empres = pd.read_excel("INE/empres.xlsx")
renmed = pd.read_excel("INE/renmed.xlsx")
desemp = pd.read_excel("INE/desemp.xlsx")
pobrez = pd.read_excel("INE/pobrez.xlsx")
nuts2_match = pd.read_excel("NUTS_v3.xlsx")

In [ ]:
application = application.merge(nuts2_match, left_on="Promotor/Distrito_D", right_on="Distrito", how="left")
application = application.merge(nuts2_match, left_on="Resumo/Distrito_D", right_on="Distrito", how="left")
application = application.rename(columns={"NUTS II_x": "nuts_ii_sede", "NUTS II_y": "nuts_ii_op"})

# FAZER O APPLICATION MERGE COM DADOS INE
application = application.merge(ginind[["NUTS II", "median"]], left_on="nuts_ii_sede", right_on="NUTS II", how="left").rename(columns={"median": "Gini_NUTS_prom"})
application = application.merge(estemp[["NUTS II", "median"]], left_on="nuts_ii_sede", right_on="NUTS II", how="left").rename(columns={"median": "estemp"})
application = application.merge(denpop[["NUTS II", "median"]], left_on="nuts_ii_sede", right_on="NUTS II", how="left").rename(columns={"median": "densidade_pop_NUTS_prom"})
application = application.merge(uninut[["NUTS II", "median"]], left_on="nuts_ii_sede", right_on="NUTS II", how="left").rename(columns={"median": "Rede_Universitaria_distrito"})
application = application.merge(empres[["NUTS II", "median"]], left_on="nuts_ii_sede", right_on="NUTS II", how="left").rename(columns={"median": "n_pme_NUTS_prom"})
application = application.merge(renmed[["NUTS II", "median"]], left_on="nuts_ii_sede", right_on="NUTS II", how="left").rename(columns={"median": "remuneracao_mensal_media_NUTS_prom"})
application = application.merge(desemp[["NUTS II", "median"]], left_on="nuts_ii_sede", right_on="NUTS II", how="left").rename(columns={"median": "Tx_desemp_NUTS_prom"})
application = application.merge(pobrez[["NUTS II", "median"]], left_on="nuts_ii_sede", right_on="NUTS II", how="left").rename(columns={"median": "Risco_pobreza_NUTS_prom"})


application["mesma_sede_nuts_ii"] = np.where(application['nuts_ii_sede'].astype(str).reset_index(drop=True).eq(application['nuts_ii_op'].astype(str).reset_index(drop=True)), 1, 0)#np.where(application['nuts_ii_sede'].astype(str) == application['nuts_ii_op'].astype(str), 1, 0)#np.select(conditions, values, default=0)

In [ ]:
# Business rule - VER MAIPRO NOVO MAP:
#Escolher a data mais antiga entre Promotor/Dt_Const
# e Promotor/Dt_Inicio_Act 1 se Parametros/Ano_Cand
# de candidaturas_geral - o ano de abertura_atividade_prom_cand for <= 4,
# nulo se um dos campos for nulo, 0 c.c.

# date format: dd-mm-yyyy

# function to correct problems with dates
def correct_year(year):
    if year > pd.datetime.now().year:
        year = year - 100
    return year

application["Promotor/Dt_Inicio_Act"] = pd.to_datetime(application['Promotor/Dt_Inicio_Act'],  dayfirst=True, format="%d-%m-%y", errors="coerce").dt.year
application["Promotor/Dt_Inicio_Act"] = application["Promotor/Dt_Inicio_Act"].apply(correct_year)
application["Promotor/Dt_Const"] = pd.to_datetime(application['Promotor/Dt_Const'], dayfirst=True, format="%d-%m-%y", errors="coerce").dt.year
application["Promotor/Dt_Const"] = application["Promotor/Dt_Const"].apply(correct_year)

application["older_date"] = application[['Promotor/Dt_Inicio_Act', 'Promotor/Dt_Const']].min(axis=1)
application["older_date_vs_4years"] = (application["Parametros/Ano_Cand"] - application['older_date'])


# Validate how to populate final date field using set of conditions, based on initial business rules
conditions = [
    (application['Promotor/Dt_Inicio_Act'].isnull()) |
    (application['Promotor/Dt_Const'].isnull()) |
    (application["Parametros/Ano_Cand"].isnull()),  # Condition for null values
    (application['older_date_vs_4years'] <= 4)  # Condition for year difference less than or equal to 4
]
values = [np.nan, 1]
application["emp_menos_4_anos_cand"] = np.select(conditions, values, default=0)

In [ ]:
application["emp_menos_4_anos_cand"]

In [ ]:
invoice = invoice.rename(columns={"ANON_Nproj": "N_Proj_anon", "N_Doc": "proj_investimento"})
invoice[invoice["N_Proj_anon"].isin(projects)]["N_Proj_anon"].nunique()

In [ ]:
# Business rule - VER MAIPRO NOVO MAP:
# Só foram selecionadas as linhas do último N_Doc; 
# só foram selecionadas as linhas onde C_An é diferente de "X",
# incluindo os nulos;  só foram selecionadas as linhas onde Tipo é "FT" ou "VE" ou "CL"

valid_inv_type = ["FT", "VE", "CL"]
invoice = invoice[(invoice["C_An"] != "X") |
       (invoice["C_An"].isna()) |
       (invoice["Tipo"].isin(valid_inv_type))].groupby("N_Proj_anon").max(["proj_investimento"])["Desp_Inv"].to_frame()

In [ ]:
worker = worker.rename(columns={"ANON_N_proj": "N_Proj_anon", "N_Linha": "N_trabalhadores"})

In [ ]:
worker[worker["N_Proj_anon"].isin(projects)]["N_Proj_anon"].nunique()

In [ ]:
worker.columns

In [ ]:
n_workers = worker.groupby("N_Proj_anon")["N_trabalhadores"].count().to_frame()

In [ ]:
financing[financing["N_Proj_anon"].isin(projects)]["N_Proj_anon"].nunique()

In [ ]:
financing.columns

In [ ]:
WeightCP = [1, 91]
WeightSelfFinance = [2, 91]
WeightRandomCapital = [4, 407, 408, 91]
WeightPartners = [403, 102, 91]
WeightTotal = [407, 408, 91 ]
WeightReimbursed = [40701, 91]
PercEligibility = [92, 91]


cols_sum = ['Val_P1', 'Val_0', 'Val_1', 'Val_2', 'Val_3', 'Val_4', 'Val_5', 'Val_6', 'Val_7']

In [ ]:
total_fin = financing[financing['Rubrica'] == 91].groupby("N_Proj_anon")[cols_sum].sum().T.sum().to_frame().reset_index()
total_fin = total_fin.rename(columns={0: "total"})

rub_1 = financing[((financing['Rubrica'] == 1))].groupby("N_Proj_anon")[cols_sum].sum().T.sum().to_frame().reset_index()
rub_1 = rub_1.rename(columns={0: "rubrica_1"})

rub_2 = financing[((financing['Rubrica'] == 2))].groupby("N_Proj_anon")[cols_sum].sum().T.sum().to_frame().reset_index()
rub_2 = rub_2.rename(columns={0: "rubrica_2"})

rub_4 = financing[((financing['Rubrica'] == 4))].groupby("N_Proj_anon")[cols_sum].sum().T.sum().to_frame().reset_index()
rub_4 = rub_4.rename(columns={0: "rubrica_4"})

rub_407 = financing[((financing['Rubrica'] == 408))].groupby("N_Proj_anon")[cols_sum].sum().T.sum().to_frame().reset_index()
rub_407 = rub_407.rename(columns={0: "rubrica_408"})

rub_408 = financing[((financing['Rubrica'] == 407))].groupby("N_Proj_anon")[cols_sum].sum().T.sum().to_frame().reset_index()
rub_408 = rub_408.rename(columns={0: "rubrica_407"})

rub_403_102 = financing[((financing['Rubrica'] == 403) | (financing['Rubrica'] == 102))].groupby("N_Proj_anon")[cols_sum].sum().T.sum().to_frame().reset_index()
rub_403_102 = rub_403_102.rename(columns={0: "rubrica_403_102"})

rub_40701 = financing[((financing['Rubrica'] == 40701))].groupby("N_Proj_anon")[cols_sum].sum().T.sum().to_frame().reset_index()
rub_40701 = rub_40701.rename(columns={0: "rubrica_40701"})

rub_92 = financing[((financing['Rubrica'] == 92))].groupby("N_Proj_anon")[cols_sum].sum().T.sum().to_frame().reset_index()
rub_92 = rub_92.rename(columns={0: "rubrica_92"})

financing_rubricas = total_fin.merge(rub_1, on="N_Proj_anon", how="left")
financing_rubricas = financing_rubricas.merge(rub_2, on="N_Proj_anon", how="left")
financing_rubricas = financing_rubricas.merge(rub_4, on="N_Proj_anon", how="left")
financing_rubricas = financing_rubricas.merge(rub_407, on="N_Proj_anon", how="left")
financing_rubricas = financing_rubricas.merge(rub_408, on="N_Proj_anon", how="left")
financing_rubricas = financing_rubricas.merge(rub_403_102, on="N_Proj_anon", how="left")
financing_rubricas = financing_rubricas.merge(rub_40701, on="N_Proj_anon", how="left")
financing_rubricas = financing_rubricas.merge(rub_92, on="N_Proj_anon", how="left")

In [ ]:
total_fin

In [ ]:
#selecionar rubricas 1 e 91, somar todos os valores Total = Financ_candidaturas['Val_P1']
# + Financ_candidaturas['Val_0'] + Financ_candidaturas['Val_1'] + Financ_candidaturas['Val_2'] 
# + Financ_candidaturas['Val_3'] + Financ_candidaturas['Val_4'] + Financ_candidaturas['Val_5'] 
# + Financ_candidaturas['Val_6'] + Financ_candidaturas['Val_7'] 1 /91
financing_rubricas["PesoCP"] = financing_rubricas["rubrica_1"]/financing_rubricas["total"]

#selecionar rubricas 2 e 91, somar todos os valores Total = Financ_candidaturas['V al_P1']
#+ Financ_candidaturas['V al_0'] + Financ_candidaturas['V al_1'] + Financ_candidaturas['V al_2']
#+ Financ_candidaturas['V al_3'] + Financ_candidaturas['V al_4'] + Financ_candidaturas['V al_5'] 
#+ Financ_candidaturas['V al_6'] + Financ_candidaturas['V al_7'] 2/91
financing_rubricas["PesoAutofinanciamento"] = financing_rubricas["rubrica_2"]/financing_rubricas["total"]

#selecionar rubricas 4, 407, 408 e 91, somar todos os valores Total = Financ_candidaturas['V al_P1']
#+ Financ_candidaturas['V al_0'] + Financ_candidaturas['V al_1'] + Financ_candidaturas['V al_2'] 
#+ Financ_candidaturas['V al_3'] + Financ_candidaturas['V al_4'] + Financ_candidaturas['V al_5'] 
#+ Financ_candidaturas['V al_6'] + Financ_candidaturas['V al_7'] se existir valor do 408: (4 - 408)/91; 
#    caso contrário: (4 - 407)/91
# Validate how to populate final date field using set of conditions, based on initial business rules
conditions = [
    ~(financing_rubricas['rubrica_408'].isnull())  # Condition for year difference less than or equal to 4
]
values = [(financing_rubricas["rubrica_4"] - financing_rubricas['rubrica_408'])/financing_rubricas["total"]]
financing_rubricas["PesoCapitais Alheios"] = np.select(conditions, values,
                                                       default=(financing_rubricas["rubrica_4"] - financing_rubricas['rubrica_407'])/financing_rubricas["total"])
        
#selecionar rubricas 403, 102 e 91, somar todos os valores Total = Financ_candidaturas['Val_P1']
#+ Financ_candidaturas['V al_0'] + Financ_candidaturas['V al_1'] + Financ_candidaturas['V al_2']
#+ Financ_candidaturas['V al_3'] + Financ_candidaturas['V al_4'] + Financ_candidaturas['V al_5']
#+ Financ_candidaturas['V al_6'] + Financ_candidaturas['V al_7'] (403 + 102)/91
financing_rubricas["Peso dos Sócios"] = (financing_rubricas["rubrica_403_102"])/financing_rubricas["total"]



#"selecionar rubricas 407, 408 e 91, somar todos os valores Total =Financ_candidaturas['Val_P1']
#+ Financ_candidaturas['Val_0'] + Financ_candidaturas['Val_1'] + Financ_candidaturas['Val_2']
#+ Financ_candidaturas['Val_3'] + Financ_candidaturas['Val_4'] + Financ_candidaturas['Val_5']
#+ Financ_candidaturas['Val_6'] + Financ_candidaturas['Val_7'] se existir valor do 408: 408/91; caso contrário: 407/91"
conditions = [
    ~(financing_rubricas['rubrica_408'].isnull())  # Condition for year difference less than or equal to 4
]
values = [financing_rubricas['rubrica_408']/financing_rubricas["total"]]
financing_rubricas["Peso Incentivo Total"] = np.select(conditions, values,
                                                       default=financing_rubricas['rubrica_407']/financing_rubricas["total"])
 
#selecionar rubricas 40701 e 91, somar todos os valores Total = Financ_candidaturas['V al_P1']
#+ Financ_candidaturas['V al_0'] + Financ_candidaturas['V al_1'] + Financ_candidaturas['V al_2']
#+ Financ_candidaturas['V al_3'] + Financ_candidaturas['V al_4'] + Financ_candidaturas['V al_5']
#+ Financ_candidaturas['V al_6'] + Financ_candidaturas['V al_7'] 40701/91
financing_rubricas["Peso Incentivo N Reembolsável"] = financing_rubricas["rubrica_40701"]/financing_rubricas["total"]


#"selecionar rubricas 92 e 91, somar todos os valores Total = Financ_candidaturas['V al_P1']
#+ Financ_candidaturas['V al_0'] + Financ_candidaturas['V al_1'] +Financ_candidaturas['Val_2']
#+ Financ_candidaturas['Val_3'] + Financ_candidaturas['Val_4'] + Financ_candidaturas['Val_5'] 
#+ Financ_candidaturas['Val_6'] + Financ_candidaturas['Val_7'] 92/91"


financing_rubricas["%elegibilidade"] = financing_rubricas["rubrica_92"]/financing_rubricas["total"]



#se algum dos rácios for superior a 1 então 1, caso contrário 0
conditions = [
    (financing_rubricas["PesoCP"] > 1) |
    (financing_rubricas["PesoAutofinanciamento"] > 1) |
    (financing_rubricas["PesoCapitais Alheios"] > 1) |
    (financing_rubricas["Peso dos Sócios"] > 1) |
    (financing_rubricas["Peso Incentivo Total"] > 1) |
    (financing_rubricas["Peso Incentivo N Reembolsável"] > 1) |
    (financing_rubricas["%elegibilidade"] > 1)
]
values = [1]
financing_rubricas["erros_racios_inv"] = np.select(conditions, values, default=0)



financing_weights = ["PesoCP", "PesoAutofinanciamento", "PesoCapitais Alheios", "Peso dos Sócios", "Peso Incentivo Total", "Peso Incentivo N Reembolsável", "%elegibilidade", "erros_racios_inv"]
financing_rubricas[financing_weights] = financing_rubricas[financing_weights].fillna(0)
financing_weights.append("N_Proj_anon")

In [ ]:
esf[esf["N_Proj_anon"].isin(projects)]["N_Proj_anon"].nunique()

In [ ]:
esf

In [ ]:
# Business Rules
# Selecionar apenas as linhas onde Id é 90
# Agregar por projeto e somar os valores de Val_Calc,
# preencher os nulos com 0, usar o QuantileTransformer (normaliza os dados entre 0 e 1)
valor_formacoes = esf[esf["Id"] == "90"].groupby("N_Proj_anon")["Val_Calc"].sum().to_frame()
valor_formacoes = valor_formacoes.rename(columns={"Val_Calc": "valor_formacoes"})

In [ ]:
expenditure[expenditure["N_Proj_anon"].isin(projects)]["N_Proj_anon"].nunique()

In [ ]:
expenditure["Classe"].dtype

In [ ]:
expenditure["N_Proj_anon"].nunique()

In [ ]:
total_proj = expenditure.groupby('N_Proj_anon', as_index=False).agg({'N_Proj_anon': 'first', 'Investimento': 'sum'})
total_proj = total_proj.rename(columns={"Investimento": "total"})
display(total_proj)

In [ ]:
# Alterado a 15/9 para refletir correção vista
soft = [230,120] # tipodesp_Software_Equipamentos_Informaticos
cons = [411,412] # tipodesp_Construcao-Remodelacao_Edificios
stud = [331,335,334,332,320,220,333] # tipodesp_Estudos_Diagnosticos_Licencas_ServicosEngenharia
mach = [110] # tipodesp_Maquinas_Equipamentos
#default - tipodesp_Outras Despesas

In [ ]:
conditions = [expenditure["Classe"].isin(soft),
             expenditure["Classe"].isin(cons),
             expenditure["Classe"].isin(stud),
             expenditure["Classe"].isin(mach)]

values = [1, 2, 3, 4]
expenditure["tipo_desp"] = np.select(conditions, values, default=9)

tipo_desp = expenditure.groupby(["N_Proj_anon", "tipo_desp"], as_index=False). \
                                    agg({'N_Proj_anon': 'first', 'tipo_desp' : "first", 'Investimento': 'sum'}). \
                                    merge(total_proj, on="N_Proj_anon")

tipo_desp["%do total"] = (tipo_desp["Investimento"]/tipo_desp["total"]).round(2)

display(tipo_desp)
tipo_desp = pd.pivot_table(tipo_desp, values = ["Investimento", "%do total", "total"], index=["N_Proj_anon"], columns = ['tipo_desp']).reset_index()


In [ ]:
tipo_desp.columns = [''.join(str(col)) for col in tipo_desp.columns.values]
new_cols = {"('N_Proj_anon', '')": "N_Proj_anon",
"('%do total', 1)": "tipodesp_Software_Equipamentos_Informaticos",
"('%do total', 2)": "tipodesp_Construcao-Remodelacao_Edificios",
"('%do total', 3)": "tipodesp_Estudos_Diagnosticos_Licencas_ServicosEngenharia",
"('%do total', 4)": "tipodesp_Maquinas_Equipamentos",
"('%do total', 9)": "tipodesp_Outras Despesas",
"('Investimento', 1)": "despesa_total_Software_Equipamentos_Informaticos",
"('Investimento', 2)": "despesa_total_Construcao/Remodelacao_Edificios",
"('Investimento', 3)": "despesa_total_Estudos_Diagnosticos_Licencas_ServicosEngenharia",
"('Investimento', 4)": "despesa_total_Maquinas_Equipamentos",
"('Investimento', 9)": "despesa_total_Outras Despesas",
"('total', 1)": "total"}

cols_drop = ["('total', 2)","('total', 3)","('total', 4)","('total', 9)"]
tipo_desp = tipo_desp.drop(cols_drop, axis=1)
tipo_desp = tipo_desp.rename(columns=new_cols)
display(tipo_desp)
tipo_desp = tipo_desp.fillna(0)

In [ ]:
ies = pd.concat([ies_1, ies_2], ignore_index=True)
#display(ies)
ies["_5001_VENDAS_SERVICOS_PRESTADOS"] = ies["_5001_VENDAS_SERVICOS_PRESTADOS"].fillna(0)
#ies[ies["_5001_VENDAS_SERVICOS_PRESTADOS"] < 0]["_5001_VENDAS_SERVICOS_PRESTADOS"] = 0

vol_negocios = ies.groupby("NIF_anon").agg({'NIF_anon': 'first', "_5001_VENDAS_SERVICOS_PRESTADOS": 'mean'}).reset_index(drop=True)#.groupby("NIF_anon")["_5001_VENDAS_SERVICOS_PRESTADOS"].median().to_frame()
vol_negocios = vol_negocios.rename(columns={"_5001_VENDAS_SERVICOS_PRESTADOS": "prom_vol_negocio_med_anual"})
ies = ies.merge(vol_negocios, how="left", on="NIF_anon")


ies = ies.rename(columns={"NIF_anon": "NIF_Prom_anon"})
ies = ies.merge(application[["N_Proj_anon",
                             "Parametros/Ano_Cand",
                             "NIF_Prom_anon",
                             "Dadosprojecto/Investimento"]],
                                how="left", on="NIF_Prom_anon")


ies["nif_ano"] = ies["NIF_Prom_anon"].astype(str).str.cat(ies["ANO_EXERCICIO"].astype(str).str).astype(float) 
display(ies[ies["N_Proj_anon"] == 595796][["N_Proj_anon", "Parametros/Ano_Cand", "ANO_EXERCICIO", "nif_ano", "prom_vol_negocio_med_anual"]])

#display(ies[ies["NIF_Prom_anon"] == 276681390][["N_Proj_anon", "Parametros/Ano_Cand", "ANO_EXERCICIO", "nif_ano", "prom_vol_negocio_med_anual"]])
ies = ies.drop_duplicates(subset=["N_Proj_anon", "nif_ano"]).reset_index(drop = True)
#display(ies[ies["NIF_Prom_anon"] == 276681390][["N_Proj_anon", "Parametros/Ano_Cand", "ANO_EXERCICIO", "nif_ano", "prom_vol_negocio_med_anual"]])
display(ies[ies["N_Proj_anon"] == 595796][["N_Proj_anon", "Parametros/Ano_Cand", "ANO_EXERCICIO", "nif_ano", "prom_vol_negocio_med_anual"]])



ies["_5127_ATIVO_TOTAL"] = ies["_5127_ATIVO_TOTAL"].fillna(0)
ies.loc[ies["_5127_ATIVO_TOTAL"] < 0, "_5127_ATIVO_TOTAL"] = 0

ies["_5143_PASSIVO_NC_FINANCIAMENTOS_OBTD"] = ies["_5143_PASSIVO_NC_FINANCIAMENTOS_OBTD"].fillna(0)
ies.loc[ies["_5143_PASSIVO_NC_FINANCIAMENTOS_OBTD"] < 0, "_5143_PASSIVO_NC_FINANCIAMENTOS_OBTD"] = 0

ies["_5113_ATIVO_COR_INVENTARIOS"] = ies["_5113_ATIVO_COR_INVENTARIOS"].fillna(0)
ies.loc[ies["_5113_ATIVO_COR_INVENTARIOS"] < 0, "_5113_ATIVO_COR_INVENTARIOS"] = 0

ies["_5114_ATIVO_COR_ACTIVOS_BIOLOGICOS"] = ies["_5114_ATIVO_COR_ACTIVOS_BIOLOGICOS"].fillna(0)
ies.loc[ies["_5114_ATIVO_COR_ACTIVOS_BIOLOGICOS"] < 0, "_5114_ATIVO_COR_ACTIVOS_BIOLOGICOS"] = 0

ies["_5115_ATIVO_COR_CLIENTES"] = ies["_5115_ATIVO_COR_CLIENTES"].fillna(0)
ies.loc[ies["_5115_ATIVO_COR_CLIENTES"] < 0, "_5115_ATIVO_COR_CLIENTES"] = 0

ies["_5116_ATIVO_COR_ADIANTAMENTOS_FORNEC"] = ies["_5116_ATIVO_COR_ADIANTAMENTOS_FORNEC"].fillna(0)
ies.loc[ies["_5116_ATIVO_COR_ADIANTAMENTOS_FORNEC"] < 0, "_5116_ATIVO_COR_ADIANTAMENTOS_FORNEC"] = 0

ies["_5117_ATIVO_COR_ESTADO_OUT_ENTES_PUB"] = ies["_5117_ATIVO_COR_ESTADO_OUT_ENTES_PUB"].fillna(0)
ies.loc[ies["_5117_ATIVO_COR_ESTADO_OUT_ENTES_PUB"] < 0, "_5117_ATIVO_COR_ESTADO_OUT_ENTES_PUB"] = 0

ies["_5148_PASSIVO_COR_FORNCEDORES"] = ies["_5148_PASSIVO_COR_FORNCEDORES"].fillna(0)
ies.loc[ies["_5148_PASSIVO_COR_FORNCEDORES"] < 0, "_5148_PASSIVO_COR_FORNCEDORES"] = 0

ies["_5149_PASSIVO_COR_ADIANTA_DE_CLIENTES"] = ies["_5149_PASSIVO_COR_ADIANTA_DE_CLIENTES"].fillna(0)
ies.loc[ies["_5149_PASSIVO_COR_ADIANTA_DE_CLIENTES"] < 0, "_5149_PASSIVO_COR_ADIANTA_DE_CLIENTES"] = 0

ies["_5150_PASSIVO_COR_ESTADO_OUT_ENT_PUB"] = ies["_5150_PASSIVO_COR_ESTADO_OUT_ENT_PUB"].fillna(0)
ies.loc[ies["_5150_PASSIVO_COR_ESTADO_OUT_ENT_PUB"] < 0, "_5150_PASSIVO_COR_ESTADO_OUT_ENT_PUB"] = 0

ies["_5025_RESULTADO_LIQUIDO_PERIODO"] = ies["_5025_RESULTADO_LIQUIDO_PERIODO"].fillna(0)
ies.loc[ies["_5025_RESULTADO_LIQUIDO_PERIODO"] < 0, "_5025_RESULTADO_LIQUIDO_PERIODO"] = 0

ies["_5141_CP_TOTAL"] = ies["_5141_CP_TOTAL"].fillna(0)
ies.loc[ies["_5141_CP_TOTAL"] < 0, "_5141_CP_TOTAL"] = 0

ies["_5139_CP_RESULTADO_LIQUIDO_PERIODO"] = ies["_5139_CP_RESULTADO_LIQUIDO_PERIODO"].fillna(0)
ies.loc[ies["_5139_CP_RESULTADO_LIQUIDO_PERIODO"] < 0, "_5139_CP_RESULTADO_LIQUIDO_PERIODO"] = 0

ies["_5127_ATIVO_TOTAL"] = ies["_5127_ATIVO_TOTAL"].fillna(0)
ies.loc[ies["_5127_ATIVO_TOTAL"] < 0, "_5127_ATIVO_TOTAL"] = 0



conditions = [ies["Parametros/Ano_Cand"].isnull(),
              ies["ANO_EXERCICIO"] == (ies["Parametros/Ano_Cand"]-1),
              ies["ANO_EXERCICIO"] == (ies["Parametros/Ano_Cand"]-2),
              ies["ANO_EXERCICIO"] == (ies["Parametros/Ano_Cand"]-3),
              #ies["ANO_EXERCICIO"] == (ies["Parametros/Ano_Cand"]),
              #ies["ANO_EXERCICIO"] == (ies["Parametros/Ano_Cand"]+1),
              #ies["ANO_EXERCICIO"] == (ies["Parametros/Ano_Cand"]+2),
]
values = [0, 1, 2, 3] #4, 5, 6]

ies["ano t-1"] = np.select(conditions, values, default=0)
ies = ies[ies["ano t-1"] >= 1]
display(ies[ies["N_Proj_anon"] == 595796][["ano t-1", "N_Proj_anon", "Parametros/Ano_Cand", "ANO_EXERCICIO", "nif_ano", "prom_vol_negocio_med_anual"]])

ies = ies.groupby("N_Proj_anon").min("ano t-1").reset_index()

display(ies[ies["N_Proj_anon"] == 595796][["ano t-1", "N_Proj_anon", "Parametros/Ano_Cand", "ANO_EXERCICIO", "nif_ano", "prom_vol_negocio_med_anual"]])

ies["prom_ativo_total_t-1"] = ies["_5127_ATIVO_TOTAL"]
ies["prom_volume_negocios_t-1"] = ies["_5001_VENDAS_SERVICOS_PRESTADOS"]
ies["prom_emprestimo_obtidos_passivo_ncor_t-1"] = ies["_5143_PASSIVO_NC_FINANCIAMENTOS_OBTD"]
ies["liquidez_geral_t-1"] = (ies["_5113_ATIVO_COR_INVENTARIOS"] +
                             ies["_5114_ATIVO_COR_ACTIVOS_BIOLOGICOS"] +
                             ies["_5115_ATIVO_COR_CLIENTES"] +
                             ies["_5116_ATIVO_COR_ADIANTAMENTOS_FORNEC"] +
                             ies["_5117_ATIVO_COR_ESTADO_OUT_ENTES_PUB"])/(ies["_5148_PASSIVO_COR_FORNCEDORES"] +
                                       ies["_5149_PASSIVO_COR_ADIANTA_DE_CLIENTES"] +
                                       ies["_5150_PASSIVO_COR_ESTADO_OUT_ENT_PUB"])
ies["rentabilidade_capitais_proprios_t-1"] = (ies["_5025_RESULTADO_LIQUIDO_PERIODO"])/(ies["_5141_CP_TOTAL"])
ies["resultadoliquido_ativo t-1"] = (ies["_5139_CP_RESULTADO_LIQUIDO_PERIODO"])/(ies["_5127_ATIVO_TOTAL"])
ies["prom_financ_vol_negocio_med_anual"] = (ies["Dadosprojecto/Investimento"])/(ies["prom_vol_negocio_med_anual"])

In [ ]:
display(ies[ies["N_Proj_anon"] == 5637109.00][["ano t-1", "Parametros/Ano_Cand", "ANO_EXERCICIO", "nif_ano", "prom_vol_negocio_med_anual"]])
display(ies[ies["N_Proj_anon"] == 5637109.00].groupby("N_Proj_anon").min("ano t-1")[["ano t-1", "Parametros/Ano_Cand", "ANO_EXERCICIO", "nif_ano", "prom_vol_negocio_med_anual"]])


# Add features

In [ ]:
df_projects = pd.DataFrame(projects, columns=["N_Proj_anon"])
cols = ["ANON_Nproj"]
df_projects = df_projects.merge(application[["NIF_Prom_anon", "Parametros/Ano_Cand", "N_Proj_anon", "Dadosprojecto/Investimento", "Resumo/Dimensao", "nuts_ii_sede", "Resumo/Cae", "emp_menos_4_anos_cand", "Gini_NUTS_prom",
    "estemp", "densidade_pop_NUTS_prom", "Rede_Universitaria_distrito", "n_pme_NUTS_prom",
    "remuneracao_mensal_media_NUTS_prom", "Tx_desemp_NUTS_prom","Risco_pobreza_NUTS_prom", "mesma_sede_nuts_ii"]], on="N_Proj_anon", how="left") 



df_projects = df_projects.merge(valor_formacoes, on="N_Proj_anon", how="left")
df_projects["valor_formacoes"] = df_projects["valor_formacoes"].fillna(0)

df_projects = df_projects.merge(n_workers, on="N_Proj_anon", how="left")
#df_projects = df_projects.merge(invoice, on="N_Proj_anon", how="left")
#merge com tipo_desp em vez de expenditure dado o que foi feito em termos de transformações
df_projects = df_projects.merge(tipo_desp, on="N_Proj_anon", how="left")
df_projects = df_projects.merge(financing_rubricas[financing_weights], on="N_Proj_anon", how="left")

conditions = [
    df_projects['N_Proj_anon'].isin(nulled["Nproj_anon"])
]
values = [1]
df_projects["proj_nulled"] = np.select(conditions, values, default=0)

df_projects["resumo_cae_2dig"] = df_projects["Resumo/Cae"].astype(str).str[:2]

cae_anulados = df_projects[df_projects["proj_nulled"] == 1].groupby("resumo_cae_2dig")["N_Proj_anon"].count().to_frame().reset_index()
cae_anulados = cae_anulados.rename(columns={"N_Proj_anon": "num_proj_anul_cae"})
df_projects = df_projects.merge(cae_anulados, on="resumo_cae_2dig", how="left")

cae_total = df_projects.groupby("resumo_cae_2dig")["N_Proj_anon"].count().to_frame().reset_index()
cae_total = cae_total.rename(columns={"N_Proj_anon": "num_proj_cae"})
df_projects = df_projects.merge(cae_total, on="resumo_cae_2dig", how="left")

nuts_ii_anulados = df_projects[df_projects["proj_nulled"] == 1].groupby("nuts_ii_sede")["N_Proj_anon"].count().to_frame().reset_index()
nuts_ii_anulados = nuts_ii_anulados.rename(columns={"N_Proj_anon": "num_proj_anul_nuts"})
df_projects = df_projects.merge(nuts_ii_anulados, on="nuts_ii_sede", how="left")

nuts_ii_total = df_projects.groupby("nuts_ii_sede")["N_Proj_anon"].count().to_frame().reset_index()
nuts_ii_total = nuts_ii_total.rename(columns={"N_Proj_anon": "num_proj_nuts"})
df_projects = df_projects.merge(nuts_ii_total, on="nuts_ii_sede", how="left")

df_projects = df_projects.merge(ies[["Parametros/Ano_Cand",
                                        "NIF_Prom_anon",
                                        "prom_vol_negocio_med_anual",
                                        "prom_ativo_total_t-1",
                                        "prom_volume_negocios_t-1",
                                        "prom_emprestimo_obtidos_passivo_ncor_t-1",
                                        "liquidez_geral_t-1",
                                        "rentabilidade_capitais_proprios_t-1",
                                        "resultadoliquido_ativo t-1",
                                        "prom_financ_vol_negocio_med_anual",
                                    "nif_ano", "N_Proj_anon"]],
                                on=["N_Proj_anon"], how="left")

df_projects["freq_target_anul_CAE"] = df_projects["num_proj_anul_cae"]/df_projects["num_proj_cae"]
df_projects["freq_target_anul_distrito"] = df_projects["num_proj_anul_nuts"]/df_projects["num_proj_nuts"]

df_projects["micro_emp"] = np.where(df_projects["Resumo/Dimensao"] == 1, 1, 0)
df_projects["pequena_emp"] = np.where(df_projects["Resumo/Dimensao"] == 2, 1, 0)
df_projects["media_emp"] = np.where(df_projects["Resumo/Dimensao"] == 3, 1, 0)
df_projects["nao_pme"] = np.where(df_projects["Resumo/Dimensao"] == 4, 1, 0)

df_projects = df_projects.drop_duplicates(subset="N_Proj_anon").reset_index(drop = True)
df_projects = df_projects.drop(["Parametros/Ano_Cand_y", "NIF_Prom_anon_y"], axis=1)


df_projects = df_projects.rename(columns={"Dadosprojecto/Investimento": "proj_investimento",
                                          "NIF_Prom_anon_x": "NIF_Prom_anon",
                                          "Parametros/Ano_Cand_x": "Parametros/Ano_Cand"})
df_projects.to_excel("IAPMEI_final_estatística_descritiva.xlsx")


#df_projects[df_projects["NIF_Prom_anon"] == 276681390]

In [ ]:
df_projects[df_projects["nao_pme"] == 0].to_excel("IAPMEI-para-flourish.xlsx")

In [ ]:
df_sem_naopme = df_projects[df_projects["nao_pme"] == 0]

In [ ]:
df_sem_naopme.groupby(["nuts_ii_sede", "proj_nulled"])["proj_investimento"].sum().to_frame()

In [ ]:
df_sem_naopme.groupby(["Resumo/Dimensao", "proj_nulled"])["proj_investimento"].sum().to_frame()

In [ ]:
df_sem_naopme.groupby(["Parametros/Ano_Cand", "proj_nulled"])["proj_investimento"].sum().to_frame()

In [ ]:
df_sem_naopme.groupby("resumo_cae_2dig")["proj_nulled"].value_counts().to_frame().to_excel("cae por cancelados.xlsx")

In [ ]:
df_projects["proj_nulled"].value_counts()

Data profiling com Não-PME vs Sem-PME

In [ ]:
df_projects[df_projects["nao_pme"] == 0].describe().T.to_excel("desc_sem_naopme.xlsx")
df_projects.describe().T.to_excel("desc_com_naopme.xlsx")

# Possibility for Outlier treatment

Experiência sem Não-PME

In [ ]:
iqr_cols =[
"proj_investimento",
"prom_vol_negocio_med_anual",
"prom_ativo_total_t-1",
"prom_volume_negocios_t-1",
"prom_emprestimo_obtidos_passivo_ncor_t-1",
"liquidez_geral_t-1",
"rentabilidade_capitais_proprios_t-1",
"resultadoliquido_ativo t-1",
"prom_financ_vol_negocio_med_anual",
"PesoCP",
"PesoAutofinanciamento",
"PesoCapitais Alheios",
"Peso dos Sócios",
"Peso Incentivo Total",
"Peso Incentivo N Reembolsável",
"valor_formacoes",
"N_trabalhadores"
]

df_projects_s_npme = df_projects[(df_projects["nao_pme"] == 0)]
# calculate the IQR for each column
q1 = df_projects_s_npme[iqr_cols].quantile(0.25)
q3 = df_projects_s_npme[iqr_cols].quantile(0.75)
iqr = q3 - q1

iqr_range = [1.5]

# define the lower and upper bounds for each column
lower_bound = q1 - (1.5 * iqr)
upper_bound = q3 + (1.5 * iqr)

# identify and mark any outliers
outliers = (df_projects_s_npme[iqr_cols] < lower_bound) | (df_projects_s_npme[iqr_cols] > upper_bound)

# For each column check the number of outliers present
for column in outliers.columns:
    print("The column ", column, " has ", outliers[outliers[column] == True][column].count(), " outliers")
    #print(dataset[iqr_cols][outliers].describe())

    
print("\n")
df_projects_s_npme.loc[outliers.any(axis=1), iqr_cols].to_excel("outliers detetados_sempme.xlsx")
df_projects_s_npme[iqr_cols] = df_projects_s_npme.loc[~outliers.any(axis=1), iqr_cols]
display(df_projects_s_npme[df_projects_s_npme["proj_nulled"] == 0].describe().T)

df_projects_s_npme.describe().T.to_excel("descoutliers_semnaopme.xlsx")
# remove the outliers from the DataFrame
#dataset[iqr_cols] = dataset.loc[~outliers.any(axis=1), iqr_cols]

In [ ]:
df_projects["proj_nulled"].value_counts()

# Remoção das variáveis não relevantes para o dataset final

In [ ]:
df_projects_final.columns

In [ ]:
df_projects_final.set_index('N_Proj_anon', inplace=True)


In [ ]:
cols_drop = ["NIF_Prom_anon",
"Parametros/Ano_Cand",
"nuts_ii_sede",
"Resumo/Dimensao",
#"N_Proj_anon",
'Resumo/Cae',
'despesa_total_Software_Equipamentos_Informaticos',
'despesa_total_Construcao/Remodelacao_Edificios',
'despesa_total_Estudos_Diagnosticos_Licencas_ServicosEngenharia',
'despesa_total_Maquinas_Equipamentos',
'despesa_total_Outras Despesas',
'total',
'resumo_cae_2dig',
'num_proj_anul_cae',
'num_proj_cae',
'num_proj_anul_nuts',
'num_proj_nuts',
"nao_pme", "nif_ano"]
df_projects_final = df_projects_final.drop(cols_drop, axis=1)



# Final scaling

In [ ]:
cols_to_scale = ["Gini_NUTS_prom",
    "estemp",
    "densidade_pop_NUTS_prom",
    "Rede_Universitaria_distrito",
    "n_pme_NUTS_prom",
    "remuneracao_mensal_media_NUTS_prom",
    "Tx_desemp_NUTS_prom",
    "Risco_pobreza_NUTS_prom",
    "N_trabalhadores",
    "proj_investimento",
    "valor_formacoes",   
    "freq_target_anul_CAE",
    "freq_target_anul_distrito",
    'prom_vol_negocio_med_anual',
    'prom_ativo_total_t-1',
    'prom_volume_negocios_t-1',
    'prom_emprestimo_obtidos_passivo_ncor_t-1',
    'liquidez_geral_t-1',
    'rentabilidade_capitais_proprios_t-1',
    'resultadoliquido_ativo t-1',
    'prom_financ_vol_negocio_med_anual',
    
]

df_projects_final = df_projects_final.fillna(0)

cols_reordered = ["Gini_NUTS_prom",
"estemp",
"densidade_pop_NUTS_prom",
"Rede_Universitaria_distrito",
"n_pme_NUTS_prom",
"remuneracao_mensal_media_NUTS_prom",
"Tx_desemp_NUTS_prom",
"Risco_pobreza_NUTS_prom",
"N_trabalhadores",
"proj_investimento",
"valor_formacoes",   
"freq_target_anul_CAE",
"freq_target_anul_distrito",
'prom_vol_negocio_med_anual',
'prom_ativo_total_t-1',
'prom_volume_negocios_t-1',
'prom_emprestimo_obtidos_passivo_ncor_t-1',
'liquidez_geral_t-1',
'rentabilidade_capitais_proprios_t-1',
'resultadoliquido_ativo t-1',
'prom_financ_vol_negocio_med_anual',
                   
"emp_menos_4_anos_cand",
"tipodesp_Software_Equipamentos_Informaticos",
"tipodesp_Construcao-Remodelacao_Edificios",
"tipodesp_Estudos_Diagnosticos_Licencas_ServicosEngenharia",
"tipodesp_Maquinas_Equipamentos",
"tipodesp_Outras Despesas",
"PesoCP",
"PesoAutofinanciamento",
"PesoCapitais Alheios",
"Peso dos Sócios",
"Peso Incentivo Total",
"Peso Incentivo N Reembolsável",
"%elegibilidade",
"erros_racios_inv",
"proj_nulled",
"mesma_sede_nuts_ii",                  
"micro_emp",
"pequena_emp",
"media_emp"]

cols = {
"proj_investimento": "Investment (€)",
"emp_menos_4_anos_cand": "Young company",
"Gini_NUTS_prom": "Gini Index NUTS",
"estemp": "Company closure NUTS",
"densidade_pop_NUTS_prom": "Population density NUTS",
"Rede_Universitaria_distrito": "College Network NUTS",
"n_pme_NUTS_prom": "Number of SME NUTS",
"remuneracao_mensal_media_NUTS_prom": "Mean salary NUTS",
"Tx_desemp_NUTS_prom": "Unemployment rate NUTS",
"Risco_pobreza_NUTS_prom": "Poverty rate NUTS",
"valor_formacoes": "Value of training",
"N_trabalhadores": "Number of workers",
"tipodesp_Software_Equipamentos_Informaticos": "Expenses - IT",
"tipodesp_Construcao-Remodelacao_Edificios": "Expenses - Civil construction",
"tipodesp_Estudos_Diagnosticos_Licencas_ServicosEngenharia": "Expenses - Engineering services",
"tipodesp_Maquinas_Equipamentos": "Expenses - Equipment",
"tipodesp_Outras Despesas": "Expenses - Other",
"PesoCP": "Weight - Equity",
"PesoAutofinanciamento": "Weight - Self-financing",
"PesoCapitais Alheios": "Weight - Foreign capital",
"Peso dos Sócios": "Weight - Partners",
"Peso Incentivo Total": "Weight - Total incentive",
"Peso Incentivo N Reembolsável": "Weight - Reimbursable incentive",
"%elegibilidade": "Eligibility (%)",
"erros_racios_inv": "Errors in weights",
"proj_nulled": "Project cancelled",
"prom_vol_negocio_med_anual": "Mean turnover",
"prom_ativo_total_t-1": "Asset",
"prom_volume_negocios_t-1": "Turnover",
"prom_emprestimo_obtidos_passivo_ncor_t-1": "Liabilities (credit)",
"liquidez_geral_t-1": "Liquidity",
"rentabilidade_capitais_proprios_t-1": "Equity",
"resultadoliquido_ativo t-1": "Net profit or loss",
"prom_financ_vol_negocio_med_anual": "Financial mean turnover",
"freq_target_anul_CAE": "Historical frequency of cancellation CAE",
"freq_target_anul_distrito": "Historical frequency of cancellation NUTS",
"mesma_sede_nuts_ii": "NUTS II of Project = NUTS II of head office",
"micro_emp": "Is micro enterprise",
"pequena_emp": "Is small company",
"media_emp": "Is medium company"}

df_projects_final = df_projects_final[cols_reordered]
df_projects_micro = df_projects_final[df_projects_final["micro_emp"] == 1]
df_projects_peq_med = df_projects_final[(df_projects_final["pequena_emp"] == 1) | (df_projects_final["media_emp"] == 1)]
# Create a transformer that scales selected columns

scaler = ColumnTransformer(transformers=[('scaler', MinMaxScaler(), cols_to_scale)], remainder='passthrough')#.set_output(transform="pandas")

# Scale the feature values of the dataframe
#df_projects_sc = scaler.fit_transform(df_projects)
df_projects_sc = pd.DataFrame(scaler.fit_transform(df_projects_final), columns=cols_reordered, index=df_projects_final.index)

df_projects_micro_sc = pd.DataFrame(scaler.fit_transform(df_projects_micro), columns=cols_reordered)
df_projects_peq_med_sc = pd.DataFrame(scaler.fit_transform(df_projects_peq_med), columns=cols_reordered)

df_projects_sc = df_projects_sc.rename(columns=cols)
df_projects_micro_sc = df_projects_micro_sc.rename(columns=cols)
df_projects_peq_med_sc = df_projects_peq_med_sc.rename(columns=cols)

df_projects_sc.to_csv("iapmei-baseline-todas-comIES-com-index.csv", index=True, sep=';', encoding="utf-8")
df_projects_micro_sc.to_csv("iapmei-baseline-micro_v3-com-index.csv", index=True, sep=';', encoding="utf-8")
df_projects_peq_med_sc.to_csv("iapmei-baseline-peq_med_v3-com-index.csv", index=True, sep=';', encoding="utf-8")
#df_projects_sc.to_excel("IAPMEI_final_modelos.xlsx")

In [ ]:
df_projects_peq_med

In [ ]:
df_projects_sc.columns